In [2]:
import pandas as pd 
import pmdarima as pm

In [6]:

from statsmodels.tools.sm_exceptions import  ValueWarning
import warnings

warnings.simplefilter(action = 'ignore', category = ValueWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
df = pd.read_csv('prepared_data.csv',parse_dates=['Date'])

In [4]:
df_coffee = df.loc[df['Product']=='Coffee',['Date','Quantity']].reset_index(drop=True)
df_tea = df.loc[df['Product']=='Tea',['Date','Quantity']].reset_index(drop=True)

In [7]:
results_aic = pm.auto_arima(df_coffee['Quantity'], trace=True, information_criterion='aic',seasonal=True)
results_aic = pm.auto_arima(df_coffee['Quantity'], trace=True, information_criterion='bic',seasonal=True)

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.38 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=1990.048, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=1973.254, Time=0.05 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=1947.687, Time=0.06 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=1988.180, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=1935.648, Time=0.11 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=1937.069, Time=0.15 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=1937.139, Time=0.16 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=1935.349, Time=0.13 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=1937.156, Time=0.24 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=inf, Time=0.18 sec
 ARIMA(0,1,2)(0,0,0)[0]             : AIC=1942.626, Time=0.07 sec

Best model:  ARIMA(0,1,2)(0,0,0)[0] intercept
Total fit time: 1.564 seconds
Performing stepwise search to minimize bic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : BIC=inf, Time=0.3